# Multipool Google Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-google.sh >> /home/haviz/multipool/ai-sa-google.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [4]:
def process_by_df(source_ds, col_name):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 
    
    # trim to 2500 character
    source_ds[col_name] = source_ds[col_name].str.slice(0,2000)

    # skip stemming
    source_ds['stemmed'] = source_ds[col_name].apply(clean_up_tag)
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        # print(len(list_text[i]))
        try:
            if len(list_text[i]) > 0:
                res = pipe( list_text[i] )
                new_list.append(res)
            else:
                res = pipe( 'kalimat netral' )
                new_list.append(res)
                zip
        except Exception as e:
            # Handle the exception (error) here
            print(f"Error processing value at index {i}: {list_text[i]}")
            print(f"Error message: {str(e)}")
            res = pipe( 'kalimat netral' )
            new_list.append(res)      
            
            continue

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
# query 400 rows of data query data from table
select_query = "select * \
from news_national_medol gr \
where id not in (select news_national_medol_id from news_national_medol_sentiment) \
order by id desc limit 500"

source_ds = execute_query(select_query)

if len(source_ds) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
# source_ds.head()

In [6]:
ds_title = source_ds
res_ds = process_by_df(ds_title,'title')

  0%|          | 0/50 [00:00<?, ?it/s]

  8%|▊         | 4/50 [00:00<00:01, 34.38it/s]

 16%|█▌        | 8/50 [00:00<00:01, 34.73it/s]

 24%|██▍       | 12/50 [00:00<00:01, 34.72it/s]

 34%|███▍      | 17/50 [00:00<00:00, 37.14it/s]

...

 74%|███████▍  | 37/50 [00:01<00:00, 36.49it/s]

 82%|████████▏ | 41/50 [00:01<00:00, 34.30it/s]

 90%|█████████ | 45/50 [00:01<00:00, 35.53it/s]

 98%|█████████▊| 49/50 [00:01<00:00, 34.31it/s]

100%|██████████| 50/50 [00:01<00:00, 35.81it/s]

In [7]:
res_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,stemmed,tokenized_tweet,Prediction,Score
0,22842,"Terkendala Cuaca, Pencarian WNA Asal Taiwan Ko...",JAKARTA - Tim SAR masih terus melakukan penc...,Rabu 13 Maret 2024 00:18 WIB,,https://www.okezone.com/,https://megapolitan.okezone.com/read/2024/03/1...,2024-03-13 05:13:23.238687,11,False,,Riyan Rizki Roshali,False,terkendala cuaca pencarian wna asal taiwan kor...,"{dilanjutkan, wna, taiwan, parikudus, pagi, ko...",Neutral,0.999700
1,22841,Polisi Amankan 4 Remaja Perang Sarung di Kota ...,BOGOR - Polisi mengamankan 4 remaja yang mela...,Rabu 13 Maret 2024 00:22 WIB,,https://www.okezone.com/,https://megapolitan.okezone.com/read/2024/03/1...,2024-03-13 05:13:23.233424,11,False,,Putra Ramadhani Astyawan,False,polisi amankan remaja perang sarung di kota bogor,"{perang, sarung, kota, amankan, remaja, polisi...",Neutral,0.999717
2,22840,"Rapat Bareng DPR, KPU Siap Beri Penjelasan Pel...",JAKARTA - Komisi Pemilihan Umum (KPU) membena...,Rabu 13 Maret 2024 00:28 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/03/13/3...,2024-03-13 05:13:23.230825,11,False,,Felldy Utama,False,rapat bareng dpr kpu siap beri penjelasan pela...,"{pemilu, kpu, bareng, penjelasan, dpr, pelaksa...",Neutral,0.999737
3,22839,Aksi Geng Motor Keroyok hingga Bacok Warga di ...,BANDUNG - Sekelompok orang yang diduga anggo...,Rabu 13 Maret 2024 00:45 WIB,,https://www.okezone.com/,https://news.okezone.com/read/2024/03/13/525/2...,2024-03-13 05:13:23.226262,11,False,,Agi Ilman,False,aksi geng motor keroyok hingga bacok warga di ...,"{motor, diamankan, keroyok, soreang, geng, aks...",Neutral,0.999613
4,22838,"Didukung Mayoritas Fraksi, Baleg Yakin RUU DKJ...",JAKARTA - Anggota Badan Legislasi (Baleg) DPR...,Rabu 13 Maret 2024 01:01 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/03/13/3...,2024-03-13 05:13:23.223574,11,False,,Felldy Utama,False,didukung mayoritas fraksi baleg yakin ruu dkj ...,"{ruu, rampung, mayoritas, fraksi, baleg, dibah...",Neutral,0.998750


In [8]:
# record result
sentiment_class = 0

# INSERT INTO public.news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES( 0, 0, 0);

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "INSERT INTO news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES(%s, %s, %s);" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


In [9]:
ds_content = source_ds
res_ds_content = process_by_df(ds_content,'content')

  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:00<00:09,  5.34it/s]

  6%|▌         | 3/50 [00:00<00:05,  8.67it/s]

  8%|▊         | 4/50 [00:00<00:05,  7.86it/s]

 12%|█▏        | 6/50 [00:00<00:04,  8.87it/s]

...

 94%|█████████▍| 47/50 [00:04<00:00,  8.95it/s]

 96%|█████████▌| 48/50 [00:05<00:00,  8.56it/s]

 98%|█████████▊| 49/50 [00:05<00:00,  8.59it/s]

100%|██████████| 50/50 [00:05<00:00,  8.36it/s]

100%|██████████| 50/50 [00:05<00:00,  9.40it/s]

In [10]:
res_ds_content.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,stemmed,tokenized_tweet,Prediction,Score
0,22842,"Terkendala Cuaca, Pencarian WNA Asal Taiwan Ko...",JAKARTA - Tim SAR masih terus melakukan penc...,Rabu 13 Maret 2024 00:18 WIB,,https://www.okezone.com/,https://megapolitan.okezone.com/read/2024/03/1...,2024-03-13 05:13:23.238687,11,False,,Riyan Rizki Roshali,False,jakarta tim sar masih terus melakukan pencaria...,"{terbatas, asha, berhasil, unsur, desiana, kep...",Neutral,0.999182
1,22841,Polisi Amankan 4 Remaja Perang Sarung di Kota ...,BOGOR - Polisi mengamankan 4 remaja yang mela...,Rabu 13 Maret 2024 00:22 WIB,,https://www.okezone.com/,https://megapolitan.okezone.com/read/2024/03/1...,2024-03-13 05:13:23.233424,11,False,,Putra Ramadhani Astyawan,False,bogor polisi mengamankan remaja yang melakukan...,"{tawuran, berhasil, hantam, wilayah, dm, klik,...",Neutral,0.997083
2,22840,"Rapat Bareng DPR, KPU Siap Beri Penjelasan Pel...",JAKARTA - Komisi Pemilihan Umum (KPU) membena...,Rabu 13 Maret 2024 00:28 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/03/13/3...,2024-03-13 05:13:23.230825,11,False,,Felldy Utama,False,jakarta komisi pemilihan umum kpu membenarkan ...,"{ri, maret, sistem, pembahasan, pusat, rakyat,...",Neutral,0.999470
3,22839,Aksi Geng Motor Keroyok hingga Bacok Warga di ...,BANDUNG - Sekelompok orang yang diduga anggo...,Rabu 13 Maret 2024 00:45 WIB,,https://www.okezone.com/,https://news.okezone.com/read/2024/03/13/525/2...,2024-03-13 05:13:23.226262,11,False,,Agi Ilman,False,bandung sekelompok orang yang diduga anggota g...,"{duga, minggu, berhasil, barang, tajam, penump...",Negative,0.681552
4,22838,"Didukung Mayoritas Fraksi, Baleg Yakin RUU DKJ...",JAKARTA - Anggota Badan Legislasi (Baleg) DPR...,Rabu 13 Maret 2024 01:01 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/03/13/3...,2024-03-13 05:13:23.223574,11,False,,Felldy Utama,False,jakarta anggota badan legislasi baleg dpr ri g...,"{pks, ri, badan, baleg, iv, maret, pembahasan,...",Neutral,0.999281


In [11]:
# record result
sentiment_class = 0

for index, row in res_ds_content.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "update news_national_medol_sentiment set sentiment_category_content = %s, score_content = %s where news_national_medol_id = %s" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
